In [ ]:
# Grouping of similar objects (unsupervised learning)

intro = """
Cluster analysis, or clustering, is an unsupervised machine learning task.

It involves automatically discovering natural grouping in data. Unlike supervised learning (like predictive modeling), 
clustering algorithms only interpret the input data and find natural groups or clusters in feature spa

"""
from io import StringIO
import datetime

In [ ]:
import pandas as pd
import sklearn
print(sklearn.__version__)

In [ ]:
import requests
key='1j5mQjg9FlUS_KtOozhBWeT6R_oMbmKkpHaYzkyXy3X8'
sheet_name='1855953019'
url=f"https://docs.google.com/spreadsheets/d/{key}/export?gid={sheet_name}&format=csv"
response = requests.get(url)
assert response.status_code == 200, 'Wrong status code'
f=response.content

print(f"read {len(f)} bytes")
df = pd.read_csv(StringIO(f.decode('UTF-8')), skiprows=4)
df =(
    df.drop(df.columns[df.columns.str.contains('Unnamed')], axis=1)
    .dropna(subset=['BRANCH'])
    .query("BRANCH.str.contains('Branch')", engine='python')
    .assign(BRANCH = lambda df: df['BRANCH'].astype(str).str[:-7])
)
df.to_csv(f'pos.{datetime.datetime.now().strftime("%Y%m%d.%H%M%S")}.csv')

In [ ]:
f='pos.20210823.101703.csv'
df = pd.read_csv(f, index_col=0)

In [ ]:
print(len(df.index), 'branches:\n', list(df.BRANCH))
print("Motions under consideration:\n", list(df.columns[1:]))

In [ ]:
rmap  = dict(zip(
    sorted(list(pd.unique(df[df.columns[1:]].values.ravel('K')))),
    [0,1,0,0, -1,-1,0,0]))
rmap

In [ ]:
df.replace(rmap, inplace=True)
df

In [ ]:
from sklearn.cluster import KMeans
X = df[df.columns[1:]]
km = KMeans(n_clusters=3)
km.fit(X)
km.predict(X)
df['cluster']= km.labels_

In [ ]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=3)
gmm.fit(X)
#df['prob'] = 
df.append = gmm.predict_proba(X)

In [ ]:
# df.sort_values(['cluster', 'BRANCH']).query('cluster < 3')[['BRANCH','cluster']]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [10,8]
x_axis = principalComponents[:, 0]
y_axis = principalComponents[:, 1]
fig, ax = plt.subplots()
ax.scatter(x_axis, y_axis, c=df.cluster)
for i, txt in enumerate(df.BRANCH):
    ax.annotate(txt, (x_axis[i], y_axis[i]))

In [ ]:
pca = PCA(n_components=3)
pcs = pca.fit_transform(X)
matplotlib.rcParams['figure.figsize'] = [10,8]
x_axis = pcs[:, 0]+5
y_axis = pcs[:, 1]+5
z_axis = pcs[:, 2]+5

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from pylab import figure
from matplotlib import pyplot

fig = figure()
ax = Axes3D(fig)
for i, txt in enumerate(df.BRANCH):
    ax.scatter(x_axis[i], y_axis[i], z_axis[i], c=['b', 'g', 'r'][df.cluster[i]])
    ax.text(x_axis[i], y_axis[i], z_axis[i],  "".join([word[0] for word in txt.split()]), size=10, zorder=1)
pyplot.show()